In [ ]:
from keras.preprocessing.image import img_to_array,load_img
import numpy as np
import glob
import os 
import cv2

from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
import imutils

class Config:
    training_DS_path = 'path to training data set folder'
    train_videos_path = 'path to training vidoes'


In [ ]:
store_image = []
train_path = Config.training_DS_path
fps = 5
train_videos = os.listdir(Config.train_videos_path)
train_images_path = train_path + '/frames'
try:
    os.makedirs(train_images_path)
except FileExistsError:
    pass

def store_inarray(image_path):
    image = load_img(image_path)
    image = img_to_array(image)
    image = cv2.resize(image, (227,227), interpolation = cv2.INTER_AREA)
    gray = 0.2989*image[:,:,0]+0.5870*image[:,:,1]+0.1140*image[:,:,2]
    store_image.append(gray)

In [ ]:
for video in train_videos:
    os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(train_videos, video, fps, train_path))
    images = os.listdir(train_images_path)
    for image in images:
        image_path=train_images_path + '/' + image
        store_inarray(image_path)

In [ ]:
store_image = np.array(store_image)
a, b, c = store_image.shape

store_image.resize(b,c,a)
store_image = (store_image-store_image.mean())/(store_image.std())
store_image = np.clip(store_image,0,1)
np.save('training.npy',store_image)

In [ ]:
stae_model = Sequential()

stae_model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
stae_model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))
stae_model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

stae_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [ ]:
training_data=np.load('training.npy')
frames = training_data.shape[2]
frames = frames-frames%10

training_data = training_data[:,:,:frames]
training_data = training_data.reshape(-1,227,227,10)
training_data = np.expand_dims(training_data,axis=4)
target_data = training_data.copy()

epochs = 15
batch_size = 1

callback_save = ModelCheckpoint("Xmodel.h5", monitor="mean_squared_error", save_best_only=True)

callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

stae_model.fit(training_data,target_data, batch_size=batch_size, epochs=epochs, callbacks = [callback_save,callback_early_stopping])
stae_model.save("Xmodel.h5")
stae_model.summary()